In [6]:
import os, csv, pandas as pd, numpy as np, webbrowser
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.MolStandardize import rdMolStandardize
import py3Dmol, ipywidgets as w
from IPython.display import display, HTML, Javascript
import webbrowser
import subprocess, sys

from urllib.parse import quote_plus
from IPython.display import HTML as DHTML


DATA_FILE = Path("/Users/tomaszszostek/PycharmProjects/REFIDD/data/processed/final_dataset.csv")
BACKUP    = DATA_FILE.with_suffix(".bak")

# ── backup i wczytanie
if not BACKUP.exists():
    DATA_FILE.rename(BACKUP)
    pd.read_csv(BACKUP).to_csv(DATA_FILE, index=False)
df = pd.read_csv(DATA_FILE)

# ── cache MolBlock (lazy) ────────────────────────────────────────────────────
mol_cache: dict[int, str] = {}
def get_molblock(i: int) -> str:
    if i in mol_cache:
        return mol_cache[i]
    mol = Chem.AddHs(Chem.MolFromSmiles(df.iloc[i].canonical_smiles))
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol, maxIters=0)
    mb = Chem.MolToMolBlock(mol)
    mol_cache[i] = mb
    return mb

# ── CSS ───────────────────────────────────────────────────────────────────────
display(HTML("""
<style>
.card{border-radius:16px;box-shadow:0 4px 8px rgba(0,0,0,.08);
      padding:14px;background:#f9fafb;width:660px;}
.header{display:flex;justify-content:space-between;align-items:center;margin:4px 0 8px}
#title{font-size:1.3rem;font-weight:600;color:#000;font-family:Helvetica,Arial}
.badge{padding:2px 8px;border-radius:8px;font-weight:600;color:#fff}
.badge.blue{background:#3b82f6}.badge.red{background:#ef4444}
.counter{font-size:.95rem;color:#374151;margin-left:12px}
.bttn{border:none;padding:6px 16px;font-size:1.2rem;
      border-radius:10px;cursor:pointer;margin:4px;
      box-shadow:0 2px 4px rgba(0,0,0,.12); transition:background .15s}
.nav{background:#cbd5e1;color:#111}             /* ciemniejsze szare */
.nav:hover{background:#a8b1c3}
.del{background:#f87171;color:#fff;font-weight:700}.del:hover{background:#ef4444}
.view{background:#34d399;color:#fff;font-weight:600}.view:hover{background:#10b981}
</style>
"""))

# ── widgets ─────────────────────────────────────────────────────────────────―
out     = w.Output()
title   = w.HTML()
badge   = w.HTML()
counter = w.HTML()

prev_b  = w.Button(icon="arrow-left",  _dom_classes=["bttn","nav"])
next_b  = w.Button(icon="arrow-right", _dom_classes=["bttn","nav"])
del_b   = w.Button(description="REMOVE", _dom_classes=["bttn","del"])
see_b   = w.Button(description="See in ChEMBL", icon="external-link",
                   _dom_classes=["bttn","view"], tooltip="open compound in ChEMBL")

head_box = w.HBox([title, badge, counter], _dom_classes=["header"])
btn_box  = w.HBox([prev_b, next_b, del_b, see_b])
card     = w.VBox([head_box, out, btn_box], _dom_classes=["card"])
display(card)

# ── viewer ───────────────────────────────────────────────────────────────────
viewer = py3Dmol.view(width=640, height=420)
state  = {"i": 0}

def render(i:int):
    if len(df)==0:
        with out: out.clear_output(wait=True); print("Dataset empty.")
        title.value = badge.value = counter.value = ""
        return
    i %= len(df); state["i"] = i
    row = df.iloc[i]
    mol_id = str(row.get("ID", "(no-ID)"))
    flag   = str(row.activity_flag).lower()
    badge_col = "blue" if flag=="active" else "red"
    title.value  = f"<span id='title'>{mol_id}</span>"
    badge.value  = f"<span class='badge {badge_col}'>{row.activity_flag}</span>"
    counter.value= f"<span class='counter'>{i+1} / {len(df)}</span>"
    viewer.removeAllModels()
    viewer.addModel(get_molblock(i), "sdf")
    viewer.setStyle({"stick":{}})
    viewer.zoomTo()
    with out:
        out.clear_output(wait=True)
        viewer.show()

def shift(d): render(state["i"]+d)
prev_b.on_click(lambda _: shift(-1))
next_b.on_click(lambda _: shift(1))

def delete(_):
    if len(df)==0: return
    idx = state["i"]
    df.drop(df.index[idx], inplace=True); df.reset_index(drop=True, inplace=True)
    df.to_csv(DATA_FILE, index=False)
    mol_cache.pop(idx, None)
    mol_cache.update({(k-1 if k>idx else k):v for k,v in list(mol_cache.items())})
    render(idx)
del_b.on_click(delete)

def _system_open(url:str):
    """Niezawodnie otwiera link w systemowej przeglądarce – niezależnie od kernela."""
    if sys.platform.startswith("darwin"):          # macOS
        subprocess.run(["open", url])
    elif sys.platform.startswith("win"):           # Windows
        os.startfile(url)                          # type: ignore
    else:                                          # Linux & WSL
        subprocess.run(["xdg-open", url])

def open_chembl(_):
    """Otwiera kartę ChEMBL lub – gdy kernel blokuje – pokazuje klikalny link."""
    if len(df) == 0:
        return

    chembl_id = str(df.iloc[state["i"]].get("ID", ""))
    if not chembl_id.upper().startswith("CHEMBL"):
        with out:
            out.clear_output(wait=True)
            print("⚠️  This record has no valid ChEMBL ID.")
        return

    url = f"https://www.ebi.ac.uk/chembl/compound_report_card/{quote_plus(chembl_id)}"
    opened = webbrowser.open_new_tab(url)

    if not opened:
        # kernel zablokował – pokaż odsyłacz jako HTML
        with out:
            out.clear_output(wait=True)
            display(DHTML(
                f'<p style="font-family:Helvetica; font-size:14px;">'
                f'Kliknij link, aby otworzyć w przeglądarce:<br>'
                f'<a href="{url}" target="_blank" style="color:#2563eb;'
                f' text-decoration:underline;">{chembl_id} na ChEMBL</a></p>'
            ))

see_b.on_click(open_chembl)
render(0)



